In [88]:
import os
import torchvision.models as models 
import torch.nn as nn
import torch
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torchvision
from glob import glob
from torchinfo import summary
import numpy as np
import torch.nn.functional as F
import torchvision.transforms as T
from sklearn.metrics import roc_curve, roc_auc_score, auc, RocCurveDisplay
from tqdm.auto import tqdm
import torchmetrics
import cv2
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
device0 = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
batch_size=1

In [155]:
#dataload

def standardize(signal):
    mean_val = sum(signal) / len(signal)
    std_dev = (sum((x - mean_val) ** 2 for x in signal) / len(signal)) ** 0.5
    standardized_signal = [(x - mean_val) / std_dev for x in signal]
    return standardized_signal
def min_max_normalize(signal):
    min_val = min(signal)
    max_val = max(signal)
    normalized_signal = [(x - min_val) / (max_val - min_val) for x in signal]
    return normalized_signal
def resized(data,N):
    M=data.size
    res=np.empty(N,data.dtype)
    carry=0
    m=0
    for n in range(N):
        sum = carry
        while m*N - n*M < M :
            sum += data[m]
            m += 1
        carry = (m-(n+1)*M/N)*data[m-1]
        sum -= carry
        res[n] = sum*N/M
    return res
def vector_magnitude_normalize(signal):
    signal_array = np.array(signal)
    magnitude = np.linalg.norm(signal_array)
    normalized_signal = signal_array / magnitude
    return normalized_signal
class CustomDataset(Dataset):
    def __init__(self, data,label ,transform=None, target_transform=None):
        self.PPG_data=data
        self.PPG_label=label
    def __len__(self):
        return len(self.PPG_data)

    def __getitem__(self, idx):
        signal_tensor=self.PPG_data[idx]
        AF_signal_label=self.PPG_label[idx]
        return signal_tensor,AF_signal_label


test_data_list=glob('../../data/nor_collection/**/*.csv')

test_data=torch.empty(len(test_data_list),1,750)
test_label=torch.empty(len(test_data_list),1)

    
for i in tqdm(range(len(test_data_list))):
    signal_csv=pd.read_csv(test_data_list[i])['PPG'].to_numpy()
    AF_signal_label=0

    signal_csv=np.array(min_max_normalize(signal_csv))
    test_data[i]=torch.tensor(resized(signal_csv,750))
    test_label[i]=torch.tensor([AF_signal_label])

test_dataset=CustomDataset(test_data ,test_label)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

  0%|          | 0/1030 [00:00<?, ?it/s]

In [156]:
class CNN1D(nn.Module):
    def __init__(self):
        super(CNN1D,self).__init__()
        self.conv1=nn.Conv1d(in_channels=1, out_channels=50, kernel_size=3,padding='same', padding_mode='replicate')
        self.relu=nn.ReLU()
        self.dropout=nn.Dropout1d(0.5)
        self.pool1=torch.nn.MaxPool1d(kernel_size=5)
        self.conv2=nn.Conv1d(in_channels=50, out_channels=50, kernel_size=3,padding='same', padding_mode='replicate')
        self.relu2=nn.ReLU()
        self.dropout2=nn.Dropout1d(0.5)
        self.pool2=torch.nn.MaxPool1d(kernel_size=5)
        self.fc1=nn.Linear(1500, 200, bias=False)
        self.fc2=nn.Linear(200, 1, bias=False)
    def forward(self,x):
        
        x=self.conv1(x)
        x=self.relu(x)
        x=self.dropout(x)
        x=self.pool1(x)
        x=self.conv2(x)
        x=self.relu2(x)
        x=self.dropout2(x)
        x=self.pool2(x)
        x = torch.flatten(x, 1)
        x=self.fc1(x)
        x=self.fc2(x)
        return x


    
accuracy = torchmetrics.Accuracy(task="binary", num_classes=1).to(device0)
model = CNN1D().to(device0)
criterion = nn.BCEWithLogitsLoss().to(device0)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)


In [157]:
model.eval()
model.load_state_dict(torch.load('../../model/Waveform/Type_normalization/CNN_1D.pt'))
m = nn.Sigmoid()
y_total=torch.empty((0,1))
predict_total=torch.empty((0,1))
with torch.no_grad():
    for x, y in test_dataloader:
        y = y.to(device0).float()
        x=x.to(device0).float()
        y_total=torch.cat([y_total,y.cpu()])
        predict = model(x).to(device0)
        predict_total=torch.cat([predict_total,m(predict).cpu()])
        acc=accuracy(predict, y)
        cost = criterion(predict, y)
        

In [159]:
a=np.array(torch.where(predict_total<0.5,1,0)).mean()
print(f'prdiction_accuracy={a*100}')

prdiction_accuracy=37.4757281553398


In [160]:
predict_total

tensor([[0.5339],
        [0.7477],
        [0.8677],
        ...,
        [0.4020],
        [0.6193],
        [0.5132]])